# AEC Compliance Agent - Tutorial 2: Calculations & Analysis

## Building Performance Analysis

This notebook demonstrates how to perform geometric calculations and circulation analysis on building data. We'll calculate room properties, find evacuation routes, and analyze building connectivity.

### What You'll Learn:

- **Geometric Calculations**: Room areas, perimeters, and spatial relationships
- **Graph Analysis**: Building connectivity and circulation patterns
- **Evacuation Routes**: Finding optimal paths to exits
- **Compliance Checking**: Verifying fire safety requirements

Let's start by importing the necessary modules and loading our example data!

In [ ]:
# Import necessary libraries
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path
import sys
from typing import List, Dict, Tuple

# Add src to path for imports
sys.path.insert(0, str(Path().absolute().parent / 'src'))

# Import our calculation modules
from calculations.geometry import (
    calculate_room_area, 
    calculate_perimeter,
    get_room_centroid,
    distance_between_points,
    rooms_are_adjacent,
    find_nearest_exit,
    get_room_bounding_box
)

from calculations.graph import CirculationGraph, PathResult
from schemas import Room, Door, Project, ProjectMetadata

# Set up matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 10

print("✅ Calculation modules imported successfully!")
print("📊 Ready for geometric and circulation analysis")

## Loading Example Building Data

Let's load the comprehensive building data we'll use for our calculations.

In [ ]:
# Load example project data
try:
    with open('../data/extracted/tutorial_example.json', 'r', encoding='utf-8') as f:
        project_data = json.load(f)
    
    print("🏗️ Project Data Loaded:")
    print(f"   Project: {project_data['metadata']['project_name']}")
    print(f"   Rooms: {len(project_data['rooms'])}")
    print(f"   Doors: {len(project_data['doors'])}")
    
except FileNotFoundError:
    print("⚠️ Generating example data first...")
    import subprocess
    import os
    
    # Change to project root and generate data
    original_dir = os.getcwd()
    os.chdir('..')
    
    result = subprocess.run(["python", "scripts/create_example_data.py", 
                            "--output", "data/extracted/tutorial_example.json"],
                           capture_output=True, text=True)
    
    os.chdir(original_dir)
    
    if result.returncode == 0:
        with open('../data/extracted/tutorial_example.json', 'r', encoding='utf-8') as f:
            project_data = json.load(f)
        print("✅ Example data generated and loaded!")
    else:
        print("❌ Failed to generate example data")
        print(result.stderr)
        raise Exception("Cannot proceed without example data")

# Convert to Room and Door objects for calculations
rooms = []
for room_data in project_data['rooms']:
    room = Room(
        id=room_data['id'],
        name=room_data['name'],
        level=room_data['level'],
        boundary=room_data['boundary'],
        use_type=room_data['use_type'],
        area=room_data.get('area'),
        occupancy_load=room_data.get('occupancy_load'),
        has_emergency_lighting=room_data.get('has_emergency_lighting', False),
        has_fire_detection=room_data.get('has_fire_detection', False),
        ceiling_height=room_data.get('ceiling_height')
    )
    rooms.append(room)

doors = []
for door_data in project_data['doors']:
    door = Door(
        id=door_data['id'],
        position=door_data['position'],
        width=door_data.get('width', 0.9),
        height=door_data.get('height', 2.1),
        door_type=door_data.get('door_type', 'single'),
        is_egress=door_data.get('is_egress', False),
        fire_rating=door_data.get('fire_rating'),
        opening_direction=door_data.get('opening_direction'),
        connected_rooms=door_data.get('connected_rooms', [])
    )
    doors.append(door)

print(f"\n📋 Converted to calculation objects:")
print(f"   Room objects: {len(rooms)}")
print(f"   Door objects: {len(doors)}")
egress_doors = [d for d in doors if d.is_egress]
print(f"   Egress doors: {len(egress_doors)}")

## Room Geometric Calculations

Let's calculate basic geometric properties for all rooms in the building.

In [ ]:
# Calculate geometric properties for all rooms
room_calculations = []

print("📐 Room Geometric Analysis:")
print("=" * 80)
print(f"{'Room ID':<8} {'Name':<20} {'Area (m²)':<10} {'Perimeter (m)':<14} {'Use Type':<15}")
print("=" * 80)

total_area = 0
total_perimeter = 0

for room in rooms:
    try:
        # Calculate area and perimeter
        calculated_area = calculate_room_area(room)
        perimeter = calculate_perimeter(room)
        centroid = get_room_centroid(room)
        bbox = get_room_bounding_box(room)
        
        # Store results
        room_calc = {
            'room': room,
            'area': calculated_area,
            'perimeter': perimeter,
            'centroid': centroid,
            'bbox': bbox,
            'width': bbox[2] - bbox[0],  # max_x - min_x
            'height': bbox[3] - bbox[1]  # max_y - min_y
        }
        room_calculations.append(room_calc)
        
        # Display results
        print(f"{room.id:<8} {room.name[:18]:<20} {calculated_area:<10.1f} {perimeter:<14.1f} {room.use_type:<15}")
        
        total_area += calculated_area
        total_perimeter += perimeter
        
    except Exception as e:
        print(f"{room.id:<8} {room.name[:18]:<20} {'ERROR':<10} {'ERROR':<14} {room.use_type:<15}")
        print(f"         Error: {str(e)}")

print("=" * 80)
print(f"{'TOTALS':<8} {'':<20} {total_area:<10.1f} {total_perimeter:<14.1f}")
print(f"\n📊 Building Summary:")
print(f"   Total Floor Area: {total_area:.1f} m²")
print(f"   Average Room Area: {total_area/len(room_calculations):.1f} m²")
print(f"   Largest Room: {max(room_calculations, key=lambda x: x['area'])['room'].name} ({max(room_calculations, key=lambda x: x['area'])['area']:.1f} m²)")
print(f"   Smallest Room: {min(room_calculations, key=lambda x: x['area'])['room'].name} ({min(room_calculations, key=lambda x: x['area'])['area']:.1f} m²)")

## Room Area Visualization

Let's create a visual comparison of room areas and their spatial arrangement.

In [ ]:
# Create room area visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Left plot: Floor plan with area-based coloring
areas = [calc['area'] for calc in room_calculations]
min_area, max_area = min(areas), max(areas)

# Normalize areas for color mapping
norm_areas = [(area - min_area) / (max_area - min_area) for area in areas]
colors = plt.cm.viridis(norm_areas)

for i, calc in enumerate(room_calculations):
    room = calc['room']
    boundary = room.boundary
    
    # Create polygon
    polygon = patches.Polygon(boundary, closed=True, 
                            facecolor=colors[i], alpha=0.7,
                            edgecolor='black', linewidth=1)
    ax1.add_patch(polygon)
    
    # Add room label with area
    centroid = calc['centroid']
    ax1.text(centroid[0], centroid[1], 
            f"{room.id}\n{calc['area']:.0f} m²",
            ha='center', va='center', fontsize=8, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.8))

# Add doors
egress_doors = [door for door in doors if door.is_egress]
regular_doors = [door for door in doors if not door.is_egress]

for door in egress_doors:
    ax1.scatter(door.position[0], door.position[1], 
               c='red', marker='s', s=100, alpha=0.9, 
               edgecolors='black', linewidth=2, label='Exit' if door == egress_doors[0] else "")

for door in regular_doors:
    ax1.scatter(door.position[0], door.position[1], 
               c='blue', marker='o', s=50, alpha=0.7,
               label='Door' if door == regular_doors[0] else "")

ax1.set_xlabel('X Coordinate (meters)', fontweight='bold')
ax1.set_ylabel('Y Coordinate (meters)', fontweight='bold')
ax1.set_title('Floor Plan - Colored by Room Area', fontweight='bold', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# Add colorbar for area scale
sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=min_area, vmax=max_area))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax1, shrink=0.8)
cbar.set_label('Room Area (m²)', fontweight='bold')

# Right plot: Area distribution
room_names = [calc['room'].name for calc in room_calculations]
areas = [calc['area'] for calc in room_calculations]

# Sort by area for better visualization
sorted_data = sorted(zip(room_names, areas), key=lambda x: x[1], reverse=True)
sorted_names, sorted_areas = zip(*sorted_data)

bars = ax2.barh(range(len(sorted_names)), sorted_areas, 
                color=plt.cm.viridis([area/max_area for area in sorted_areas]),
                alpha=0.8, edgecolor='black', linewidth=0.5)

ax2.set_yticks(range(len(sorted_names)))
ax2.set_yticklabels([name[:15] + '...' if len(name) > 15 else name for name in sorted_names])
ax2.set_xlabel('Area (m²)', fontweight='bold')
ax2.set_title('Room Areas - Largest to Smallest', fontweight='bold', fontsize=14)
ax2.grid(True, alpha=0.3, axis='x')

# Add value labels on bars
for i, (bar, area) in enumerate(zip(bars, sorted_areas)):
    ax2.text(area + max_area*0.01, i, f'{area:.0f}', 
            va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print("🎨 Room area analysis complete!")
print("📊 Left: Floor plan colored by room area")
print("📊 Right: Room areas ranked from largest to smallest")

## Building Connectivity Analysis

Now let's create a circulation graph to analyze how rooms are connected and find optimal paths.

In [ ]:
# Create project object for graph analysis
metadata = ProjectMetadata(
    project_name=project_data['metadata']['project_name'],
    project_id=project_data['metadata']['project_id'],
    level_name=project_data['metadata']['level_name'],
    level_number=project_data['metadata']['level_number'],
    extraction_date=project_data['metadata']['extraction_date'],
    source_file=project_data['metadata']['source_file'],
    building_use=project_data['metadata']['building_use']
)

project = Project(
    metadata=metadata,
    levels=[],  # We'll work with individual rooms/doors for simplicity
    rooms=rooms,
    doors=doors,
    walls=[],
    fire_equipment=[],
    sectors=[],
    evacuation_routes=[]
)

# Create circulation graph
print("🔗 Building Circulation Graph:")
print("Creating graph representation...")

try:
    circulation_graph = CirculationGraph(project)
    
    # Get graph statistics
    stats = circulation_graph.get_graph_statistics()
    
    print("\n📊 Graph Statistics:")
    print("=" * 40)
    for key, value in stats.items():
        if key != 'error':
            if isinstance(value, float):
                print(f"   {key.replace('_', ' ').title()}: {value:.2f}")
            else:
                print(f"   {key.replace('_', ' ').title()}: {value}")
    
    # Check connectivity
    components = circulation_graph.get_connected_components()
    print(f"\n🔗 Connectivity Analysis:")
    print(f"   Connected Components: {len(components)}")
    
    if len(components) > 1:
        print("   ⚠️ Building has disconnected areas:")
        for i, component in enumerate(components):
            print(f"      Component {i+1}: {list(component)}")
    else:
        print("   ✅ All rooms are connected")
    
    # Find articulation points (critical for connectivity)
    articulation_points = circulation_graph.find_articulation_points()
    if articulation_points:
        print(f"\n🎯 Critical Rooms (Articulation Points):")
        for room_id in articulation_points:
            room_name = next((r.name for r in rooms if r.id == room_id), room_id)
            print(f"   {room_id}: {room_name}")
        print("   ⚠️ These rooms are critical for building connectivity")
    else:
        print("\n✅ No single points of failure in circulation")
        
except Exception as e:
    print(f"❌ Error creating circulation graph: {str(e)}")
    circulation_graph = None

## Evacuation Route Analysis

Let's find the shortest evacuation routes from each room to the nearest exit.

In [ ]:
if circulation_graph:
    # Calculate all evacuation routes
    print("🚨 Evacuation Route Analysis:")
    print("=" * 70)
    
    evacuation_routes = circulation_graph.all_evacuation_routes()
    
    # Sort by distance for analysis
    sorted_routes = sorted(evacuation_routes.items(), 
                          key=lambda x: x[1].distance)
    
    print(f"{'Room ID':<8} {'Room Name':<20} {'Distance (m)':<12} {'Path Length':<12} {'Doors Used':<10}")
    print("=" * 70)
    
    total_distance = 0
    successful_routes = 0
    
    for room_id, route in sorted_routes:
        room = next((r for r in rooms if r.id == room_id), None)
        room_name = room.name if room else room_id
        
        if route.distance < float('inf'):
            print(f"{room_id:<8} {room_name[:18]:<20} {route.distance:<12.1f} {len(route.path):<12} {len(route.doors_used):<10}")
            total_distance += route.distance
            successful_routes += 1
        else:
            print(f"{room_id:<8} {room_name[:18]:<20} {'NO PATH':<12} {'N/A':<12} {'N/A':<10}")
    
    if successful_routes > 0:
        avg_distance = total_distance / successful_routes
        print("=" * 70)
        print(f"\n📊 Evacuation Statistics:")
        print(f"   Successful Routes: {successful_routes}/{len(evacuation_routes)}")
        print(f"   Average Evacuation Distance: {avg_distance:.1f} m")
        print(f"   Longest Evacuation Distance: {max([r.distance for r in evacuation_routes.values() if r.distance < float('inf')]):.1f} m")
        print(f"   Shortest Evacuation Distance: {min([r.distance for r in evacuation_routes.values() if r.distance < float('inf')]):.1f} m")
        
        # Check compliance (typical max 25m for protected routes, 35m for unprotected)
        max_distance = max([r.distance for r in evacuation_routes.values() if r.distance < float('inf')])
        if max_distance <= 25:
            print("   ✅ All evacuation distances within protected route limit (25m)")
        elif max_distance <= 35:
            print("   ⚠️ Some routes exceed protected limit but within unprotected limit (35m)")
        else:
            print("   ❌ Some evacuation routes exceed regulatory limits")
    
    # Find the longest evacuation route for visualization
    longest_route_room = max(evacuation_routes.items(), 
                           key=lambda x: x[1].distance if x[1].distance < float('inf') else 0)
    
    print(f"\n🔍 Longest Evacuation Route:")
    print(f"   From: {longest_route_room[0]} ({next((r.name for r in rooms if r.id == longest_route_room[0]), 'Unknown')})")
    print(f"   Distance: {longest_route_room[1].distance:.1f} m")
    print(f"   Path: {' → '.join(longest_route_room[1].path)}")
    print(f"   Doors: {', '.join(longest_route_room[1].doors_used)}")
    
else:
    print("❌ Cannot analyze evacuation routes without circulation graph")

## Circulation Graph Visualization

Let's visualize the building's circulation network and highlight evacuation routes.

In [ ]:
if circulation_graph:
    # Create circulation graph visualization
    print("🎨 Generating circulation graph visualization...")
    
    # First, show the basic graph
    fig = circulation_graph.visualize_graph(figsize=(16, 12))
    plt.suptitle('Building Circulation Network', fontsize=16, fontweight='bold', y=0.95)
    plt.show()
    
    # Then show with longest evacuation route highlighted
    if 'longest_route_room' in locals():
        longest_path = longest_route_room[1].path
        fig = circulation_graph.visualize_graph(figsize=(16, 12), highlight_path=longest_path)
        plt.suptitle(f'Longest Evacuation Route: {longest_route_room[1].distance:.1f}m', 
                    fontsize=16, fontweight='bold', y=0.95)
        plt.show()
    
    print("✅ Circulation graph visualization complete!")
    print("📊 The graphs show:")
    print("   - Nodes represent rooms (colored by type)")
    print("   - Edges represent doors/connections")
    print("   - Green edges indicate egress doors")
    print("   - Red path shows longest evacuation route")
    
else:
    print("❌ Cannot visualize without circulation graph")

## Fire Equipment Coverage Analysis

Let's calculate which areas are covered by fire equipment based on their positions.

In [ ]:
# Load fire equipment data if available
fire_equipment = project_data.get('fire_equipment', [])

if fire_equipment:
    print("🔥 Fire Equipment Coverage Analysis:")
    print("=" * 60)
    
    # Calculate which rooms are covered by each type of equipment
    equipment_types = {}
    room_coverage = {room.id: {'extinguisher': False, 'hydrant': False, 'alarm': False} for room in rooms}
    
    for equipment in fire_equipment:
        eq_type = equipment['equipment_type']
        eq_pos = equipment['position']
        coverage_radius = equipment.get('coverage_radius', 15.0)
        
        if eq_type not in equipment_types:
            equipment_types[eq_type] = []
        equipment_types[eq_type].append(equipment)
        
        # Check which rooms are within coverage
        for room_calc in room_calculations:
            room_centroid = room_calc['centroid']
            distance = distance_between_points(eq_pos, room_centroid)
            
            if distance <= coverage_radius:
                if eq_type in room_coverage[room_calc['room'].id]:
                    room_coverage[room_calc['room'].id][eq_type] = True
    
    # Summary by equipment type
    for eq_type, equipment_list in equipment_types.items():
        print(f"\n{eq_type.replace('_', ' ').title()}:")
        print(f"   Count: {len(equipment_list)}")
        
        covered_rooms = sum(1 for coverage in room_coverage.values() 
                          if coverage.get(eq_type, False))
        coverage_percentage = (covered_rooms / len(rooms)) * 100
        print(f"   Room Coverage: {covered_rooms}/{len(rooms)} ({coverage_percentage:.1f}%)")
        
        # Average coverage radius
        avg_radius = sum(eq.get('coverage_radius', 15.0) for eq in equipment_list) / len(equipment_list)
        print(f"   Average Coverage Radius: {avg_radius:.1f} m")
    
    # Room-by-room coverage analysis
    print(f"\n📋 Room Coverage Details:")
    print(f"{'Room ID':<8} {'Room Name':<20} {'Extinguisher':<12} {'Hydrant':<8} {'Alarm':<8} {'Score':<6}")
    print("=" * 70)
    
    for room in rooms:
        coverage = room_coverage[room.id]
        score = sum(1 for covered in coverage.values() if covered)
        
        ext_status = "✅" if coverage.get('extinguisher', False) else "❌"
        hyd_status = "✅" if coverage.get('hydrant', False) else "❌"
        alarm_status = "✅" if coverage.get('alarm', False) else "❌"
        
        print(f"{room.id:<8} {room.name[:18]:<20} {ext_status:<12} {hyd_status:<8} {alarm_status:<8} {score}/3")
    
    # Overall coverage statistics
    total_covered = sum(1 for coverage in room_coverage.values() 
                       if any(coverage.values()))
    fully_covered = sum(1 for coverage in room_coverage.values() 
                       if all(coverage.values()))
    
    print("=" * 70)
    print(f"\n📊 Overall Coverage Statistics:")
    print(f"   Rooms with any coverage: {total_covered}/{len(rooms)} ({(total_covered/len(rooms)*100):.1f}%)")
    print(f"   Rooms with full coverage: {fully_covered}/{len(rooms)} ({(fully_covered/len(rooms)*100):.1f}%)")
    
    if total_covered == len(rooms):
        print("   ✅ All rooms have some fire protection coverage")
    else:
        uncovered = len(rooms) - total_covered
        print(f"   ⚠️ {uncovered} rooms lack fire protection coverage")
        
else:
    print("⚠️ No fire equipment data available for coverage analysis")

## Interactive Room Analysis

Let's create an interactive analysis where you can explore specific rooms and their properties.

In [ ]:
# Interactive room analysis function
def analyze_room_details(room_id: str):
    """Detailed analysis of a specific room."""
    
    # Find the room
    room = next((r for r in rooms if r.id == room_id), None)
    if not room:
        print(f"❌ Room {room_id} not found!")
        return
    
    # Get calculation data
    room_calc = next((calc for calc in room_calculations if calc['room'].id == room_id), None)
    
    print(f"🏠 Detailed Analysis: {room.name} ({room_id})")
    print("=" * 60)
    
    # Basic properties
    print(f"Use Type: {room.use_type.replace('_', ' ').title()}")
    print(f"Level: {room.level}")
    if room_calc:
        print(f"Area: {room_calc['area']:.1f} m²")
        print(f"Perimeter: {room_calc['perimeter']:.1f} m")
        print(f"Dimensions: {room_calc['width']:.1f} × {room_calc['height']:.1f} m")
        print(f"Centroid: ({room_calc['centroid'][0]:.1f}, {room_calc['centroid'][1]:.1f})")
    
    if hasattr(room, 'occupancy_load') and room.occupancy_load:
        print(f"Occupancy Load: {room.occupancy_load} people")
        if room_calc:
            density = room.occupancy_load / room_calc['area']
            print(f"Occupancy Density: {density:.2f} people/m²")
    
    if hasattr(room, 'ceiling_height') and room.ceiling_height:
        print(f"Ceiling Height: {room.ceiling_height} m")
        if room_calc:
            volume = room_calc['area'] * room.ceiling_height
            print(f"Volume: {volume:.1f} m³")
    
    # Safety features
    print(f"\n🔒 Safety Features:")
    print(f"Emergency Lighting: {'✅' if getattr(room, 'has_emergency_lighting', False) else '❌'}")
    print(f"Fire Detection: {'✅' if getattr(room, 'has_fire_detection', False) else '❌'}")
    
    # Adjacent rooms
    print(f"\n🔗 Adjacent Rooms:")
    adjacent_rooms = []
    for other_room in rooms:
        if other_room.id != room.id:
            try:
                if rooms_are_adjacent(room, other_room):
                    adjacent_rooms.append(other_room)
            except:
                pass  # Skip problematic room pairs
    
    if adjacent_rooms:
        for adj_room in adjacent_rooms:
            print(f"   - {adj_room.id}: {adj_room.name}")
    else:
        print("   No adjacent rooms detected")
    
    # Evacuation analysis
    if circulation_graph and room_id in evacuation_routes:
        route = evacuation_routes[room_id]
        print(f"\n🚨 Evacuation Route:")
        if route.distance < float('inf'):
            print(f"   Distance to Exit: {route.distance:.1f} m")
            print(f"   Path: {' → '.join(route.path)}")
            print(f"   Doors Used: {', '.join(route.doors_used) if route.doors_used else 'None'}")
        else:
            print("   ❌ No evacuation route available")
    
    # Nearest exit
    if room_calc:
        nearest_exit = find_nearest_exit(room_calc['centroid'], doors)
        if nearest_exit:
            exit_distance = distance_between_points(room_calc['centroid'], 
                                                  (nearest_exit.position[0], nearest_exit.position[1]))
            print(f"\n🚪 Nearest Exit:")
            print(f"   Door: {nearest_exit.id}")
            print(f"   Direct Distance: {exit_distance:.1f} m")
    
    # Fire equipment coverage
    if room_id in room_coverage:
        coverage = room_coverage[room_id]
        print(f"\n🔥 Fire Protection Coverage:")
        for eq_type, is_covered in coverage.items():
            status = "✅" if is_covered else "❌"
            print(f"   {eq_type.replace('_', ' ').title()}: {status}")

# Demonstrate with a few example rooms
print("🔍 Interactive Room Analysis Examples:")
print("\n" + "="*80)

# Analyze the largest room
largest_room = max(room_calculations, key=lambda x: x['area'])['room']
analyze_room_details(largest_room.id)

print("\n" + "="*80)

# Analyze a corridor (important for circulation)
corridor_room = next((r for r in rooms if r.use_type == 'corridor'), None)
if corridor_room:
    analyze_room_details(corridor_room.id)

print("\n💡 Try analyzing other rooms by calling: analyze_room_details('ROOM_ID')")
print(f"Available rooms: {', '.join([r.id for r in rooms[:10]])}{'...' if len(rooms) > 10 else ''}")

## Summary & Compliance Assessment

Let's create a comprehensive compliance report based on our calculations.

In [ ]:
# Generate comprehensive compliance report
print("📋 AEC COMPLIANCE ASSESSMENT REPORT")
print("=" * 80)
print(f"Project: {project_data['metadata']['project_name']}")
print(f"Analysis Date: {project_data['metadata']['extraction_date'][:10]}")
print(f"Building Use: {project_data['metadata']['building_use'].title()}")
print("=" * 80)

# Building metrics
print(f"\n🏢 BUILDING METRICS:")
print(f"   Total Floor Area: {total_area:.1f} m²")
print(f"   Number of Rooms: {len(rooms)}")
print(f"   Number of Doors: {len(doors)}")
print(f"   Number of Exits: {len([d for d in doors if d.is_egress])}")

# Circulation assessment
print(f"\n🔗 CIRCULATION ASSESSMENT:")
if circulation_graph:
    stats = circulation_graph.get_graph_statistics()
    print(f"   Building Connectivity: {'✅ Connected' if stats.get('is_connected', False) else '❌ Disconnected'}")
    print(f"   Average Path Length: {stats.get('average_path_length', 0):.1f}")
    print(f"   Egress Connectivity: {stats.get('egress_connectivity_ratio', 0)*100:.1f}%")
else:
    print("   ❌ Unable to assess circulation")

# Evacuation assessment
print(f"\n🚨 EVACUATION ASSESSMENT:")
if 'evacuation_routes' in locals() and evacuation_routes:
    successful_routes = sum(1 for route in evacuation_routes.values() if route.distance < float('inf'))
    if successful_routes == len(rooms):
        print("   Route Availability: ✅ All rooms have evacuation routes")
    else:
        print(f"   Route Availability: ⚠️ {len(rooms) - successful_routes} rooms lack evacuation routes")
    
    valid_distances = [r.distance for r in evacuation_routes.values() if r.distance < float('inf')]
    if valid_distances:
        max_distance = max(valid_distances)
        avg_distance = sum(valid_distances) / len(valid_distances)
        print(f"   Maximum Evacuation Distance: {max_distance:.1f} m")
        print(f"   Average Evacuation Distance: {avg_distance:.1f} m")
        
        # Compliance check (Spanish CTE DB SI typical limits)
        if max_distance <= 25:
            print("   Distance Compliance: ✅ Within protected route limits")
        elif max_distance <= 35:
            print("   Distance Compliance: ⚠️ Exceeds protected limits but within general limits")
        else:
            print("   Distance Compliance: ❌ Exceeds regulatory limits")
else:
    print("   ❌ Unable to assess evacuation routes")

# Fire protection assessment
print(f"\n🔥 FIRE PROTECTION ASSESSMENT:")
if fire_equipment and 'room_coverage' in locals():
    total_coverage = sum(1 for coverage in room_coverage.values() if any(coverage.values()))
    full_coverage = sum(1 for coverage in room_coverage.values() if all(coverage.values()))
    
    print(f"   Equipment Count: {len(fire_equipment)} units")
    print(f"   Room Coverage: {total_coverage}/{len(rooms)} ({(total_coverage/len(rooms)*100):.1f}%)")
    print(f"   Full Protection: {full_coverage}/{len(rooms)} ({(full_coverage/len(rooms)*100):.1f}%)")
    
    if total_coverage == len(rooms):
        print("   Coverage Compliance: ✅ All rooms have fire protection")
    else:
        print(f"   Coverage Compliance: ⚠️ {len(rooms) - total_coverage} rooms lack protection")
else:
    print("   ❌ Unable to assess fire protection")

# Room size compliance
print(f"\n📐 ROOM SIZE COMPLIANCE:")
oversized_rooms = [calc for calc in room_calculations if calc['area'] > 2500]  # Example limit
undersized_rooms = [calc for calc in room_calculations if calc['area'] < 5]     # Example minimum

if oversized_rooms:
    print(f"   Oversized Rooms: ⚠️ {len(oversized_rooms)} rooms exceed 2500 m²")
    for calc in oversized_rooms[:3]:  # Show first 3
        print(f"      {calc['room'].id}: {calc['area']:.1f} m²")
else:
    print("   Room Sizes: ✅ All rooms within reasonable limits")

# Overall compliance score
print(f"\n🎯 OVERALL COMPLIANCE SCORE:")
score_items = []

# Connectivity (25 points)
if circulation_graph and circulation_graph.get_graph_statistics().get('is_connected', False):
    score_items.append(25)
else:
    score_items.append(0)

# Evacuation routes (35 points)
if 'evacuation_routes' in locals() and evacuation_routes:
    successful_routes = sum(1 for route in evacuation_routes.values() if route.distance < float('inf'))
    evacuation_score = (successful_routes / len(rooms)) * 35
    score_items.append(evacuation_score)
else:
    score_items.append(0)

# Fire protection (25 points)
if 'room_coverage' in locals():
    total_coverage = sum(1 for coverage in room_coverage.values() if any(coverage.values()))
    protection_score = (total_coverage / len(rooms)) * 25
    score_items.append(protection_score)
else:
    score_items.append(0)

# Data completeness (15 points)
data_score = 15  # We have complete room and door data
score_items.append(data_score)

total_score = sum(score_items)

print(f"   Connectivity: {score_items[0]:.0f}/25")
print(f"   Evacuation: {score_items[1]:.0f}/35")
print(f"   Fire Protection: {score_items[2]:.0f}/25")
print(f"   Data Quality: {score_items[3]:.0f}/15")
print(f"   " + "="*30)
print(f"   TOTAL SCORE: {total_score:.0f}/100")

if total_score >= 90:
    grade = "✅ EXCELLENT"
elif total_score >= 75:
    grade = "✅ GOOD"
elif total_score >= 60:
    grade = "⚠️ ACCEPTABLE"
else:
    grade = "❌ NEEDS IMPROVEMENT"

print(f"   COMPLIANCE GRADE: {grade}")

print(f"\n" + "="*80)
print("📝 RECOMMENDATIONS:")
recommendations = []

if not circulation_graph or not circulation_graph.get_graph_statistics().get('is_connected', False):
    recommendations.append("• Improve building connectivity - some areas are isolated")

if 'evacuation_routes' in locals() and evacuation_routes:
    failed_routes = sum(1 for route in evacuation_routes.values() if route.distance >= float('inf'))
    if failed_routes > 0:
        recommendations.append(f"• Provide evacuation routes for {failed_routes} inaccessible rooms")
    
    valid_distances = [r.distance for r in evacuation_routes.values() if r.distance < float('inf')]
    if valid_distances and max(valid_distances) > 35:
        recommendations.append("• Reduce maximum evacuation distances or provide protected routes")

if 'room_coverage' in locals():
    uncovered = sum(1 for coverage in room_coverage.values() if not any(coverage.values()))
    if uncovered > 0:
        recommendations.append(f"• Install fire protection equipment in {uncovered} uncovered rooms")

if not recommendations:
    recommendations.append("• Building meets basic compliance requirements")
    recommendations.append("• Consider periodic review and updates as regulations change")

for rec in recommendations:
    print(rec)

print(f"\n" + "="*80)
print("✅ Compliance assessment complete!")

## Summary

In this tutorial, we've learned how to:

### 🔧 **Geometric Calculations**
- Calculate room areas and perimeters using the Shapely library
- Find room centroids and bounding boxes
- Analyze spatial relationships between rooms

### 🕸️ **Graph-Based Analysis**
- Create circulation graphs from building data
- Analyze building connectivity and identify critical paths
- Find articulation points that are crucial for circulation

### 🚨 **Evacuation Planning**
- Calculate shortest evacuation routes from each room
- Analyze evacuation distances for compliance
- Visualize circulation patterns and critical routes

### 🔥 **Fire Safety Analysis**
- Assess fire equipment coverage across the building
- Identify areas lacking adequate protection
- Calculate equipment effectiveness and placement

### 📊 **Compliance Assessment**
- Generate comprehensive compliance reports
- Score buildings against regulatory requirements
- Provide actionable recommendations for improvements

### Key Takeaways:

1. **Automation is Key**: These calculations would be tedious by hand but are instant with code
2. **Visual Analysis**: Graphs and floor plans reveal patterns not obvious in raw data
3. **Compliance Checking**: Systematic analysis ensures nothing is missed
4. **Optimization Opportunities**: Analysis reveals where improvements can be made

### Next Steps:

With these calculation tools, you can:
- Analyze your own building projects
- Customize compliance criteria for different regulations
- Integrate with CAD software for automated analysis
- Develop specialized tools for specific building types

**Want to explore more?** Try running `analyze_room_details()` with different room IDs to dive deeper into specific areas!